In [1]:
from pigeonXT import annotate

## Multi-label Classification

In [ ]:
df = pd.DataFrame([
    {'text': 'Spicy Chinese Eggplant with Szechuan Sauce with chilies and peanuts– serve or black rice or rice noodles for a tasty, easy vegan dinner!'},
    {'text': 'Spicy Chinese Eggplant with Szechuan Sauce with chilies and peanuts– serve or black rice or rice noodles for a tasty, easy vegan dinner!'},
    {'text': 'Spicy Chinese Eggplant with Szechuan Sauce with chilies and peanuts– serve or black rice or rice noodles for a tasty, easy vegan dinner!'},    
])

In [ ]:
def setLabels(labels, numClasses):
    row = np.zeros([numClasses], dtype=np.uint8)
    row[labels] = 1
    return row

def labelPortion(inputFile, 
                 labels = ['yes', 'no'], 
                 outputFile='output.parquet', 
                 portionSize=50,
                 textColumn='text',
                 shortLabels=None):
    if shortLabels == None:
        shortLabels = labels
    out = Path(outputFile)
    if out.exists():
        outdf = pd.read_parquet(out)
        currentId = outdf.index.max() + 1
    else:
        currentId = 0
    indf = pd.read_parquet(inputFile)
    examplesInFile = len(indf)
    indf = indf.loc[currentId:currentId + portionSize - 1]
    print(f'{currentId + 1} - {currentId + portionSize} of {examplesInFile}')
    sentences = indf[textColumn].tolist()
    
    for label in shortLabels:
        indf[label] = None
    
    def updateRow(ix, selectedLabels):
        labs = setLabels([labels.index(y) for y in selectedLabels], len(labels))
        indf.loc[indf.index.min()+ix, shortLabels] = labs
    
    def finalProcessing(annotations):
        if out.exists():
            prevdata = pd.read_parquet(out)
            outdata = pd.concat([prevdata, indf]).reset_index(drop=True)
        else:
            outdata = indf.copy()
        outdata.to_parquet(out, index=False)
    
    annotated = annotate( sentences, 
                          options=labels, 
                          task_type='multilabel-classification',
                          buttons_in_a_row=3,
                          reset_buttons_after_click=True,
                          include_skip=False,
                          example_process_fn=updateRow,
                          final_process_fn=finalProcessing)     
    return indf
